# Modified Q-Learning algorithm for maze solving 

## Problem   
This code implements a maze solving robot who should be able to reach the end point when its placed at any starting point.
The difference is this model is trained using already known paths.

## Terms used

The maze is given in terms of coordinates of the maze. The maze can traversed in 4 directions.

UP - 0, DOWN - 1, LEFT - 2, RIGHT - 3

Episodes: are the list of pairs made up of the starting point and the actions taken by the agent to reach the goal.

Horizontal Path List: is a list of pairs that contains the longest horizontal paths in the maze.

Vertical Path List: is a list of pairs that contains the longest vertical paths in the maze.


NOTE: for the sake of simplifying the maze, (1, 1) is added to the coordinates of the maze. 


## Importing the libraries


In [ ]:
import numpy as np
import pylab as plt
import networkx as nx

## Declaring the variables

In [ ]:
MATRIX_SIZE = 9

start = (2, 2)
finish = (4, 8)

episodes = [
    [(2,2), [0, 3, 3, 3]],
    [(6,2), [1,3,0,3,1,3]],
    [(6,4), [1,3,3]]
]

horizontal_path_list = [
    [(0,0),(0,6)],
    [(6,0),(6,8)],
    [(8,0),(8,2)], 
    [(4,2),(4,8)]
]

vertical_path_list = [
    [(0,0),(8,0)],
    [(2,2),(8,2)],
    [(4,4),(6,4)],
    [(0,6),(6,6)],
    [(6,8),(8,8)]
]


## Initializing the Maze


In [ ]:
maze = np.matrix(np.ones(shape = (MATRIX_SIZE, MATRIX_SIZE)))
maze *= -1

## Updating viable paths

In [ ]:
 #  Given Maze                     Flipped maze for programming ease(Target)    
    # 0 1 2 3 4 5 6 7 8         #  0 1 2 3 4 5 6 7 8
#   8 . . . * * * * * .         0  . . . . . . . * * 
#   7 . * . * * * * * .         1  . * * * * * . * * 
#   6 . . . . . . . . .         2  . * . * * * . * * 
#   5 . * . * . * . * *         3  . * . * * * . * * 
#   4 . * . . . . . . .         4  . * . . . . . . . 
#   3 . * . * * * . * *         5  . * . * . * . * * 
#   2 . * . * * * . * *         6  . . . . . . . . . 
#   1 . * * * * * . * *         7  . * . * * * * * . 
#   0 . . . . . . . * *         8  . . . * * * * * . 

In [ ]:
print("Creating the maze...")
for path in horizontal_path_list:
    for i in range(path[0][1], path[1][1]+1):
        maze[path[0][0], i] = 0

for path in vertical_path_list:
    for i in range(path[0][0], path[1][0]+1):
        maze[i, path[0][1]] = 0

maze[finish[0], finish[1]] = 100
print("\nMaze: \n{}\n".format(maze))

Creating the maze...

Maze: 
[[  0.   0.   0.   0.   0.   0.   0.  -1.  -1.]
 [  0.  -1.  -1.  -1.  -1.  -1.   0.  -1.  -1.]
 [  0.  -1.   0.  -1.  -1.  -1.   0.  -1.  -1.]
 [  0.  -1.   0.  -1.  -1.  -1.   0.  -1.  -1.]
 [  0.  -1.   0.   0.   0.   0.   0.   0. 100.]
 [  0.  -1.   0.  -1.   0.  -1.   0.  -1.  -1.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.  -1.   0.  -1.  -1.  -1.  -1.  -1.   0.]
 [  0.   0.   0.  -1.  -1.  -1.  -1.  -1.   0.]]



## Defining the functions

### get_max() for training the model

In [ ]:
def get_max(getFrom, x, y):
    values = []
  # 0: UP
    if y == MATRIX_SIZE-1 or maze[x, y+1] == -1:
        values.append(-1)
    else:
        values.append(getFrom[x, y+2])
  # 1: DOWN
    if y == 0 or maze[x, y-1] == -1:
        values.append(-1)
    else:
        values.append(getFrom[x, y-2])
  # 2: LEFT
    if x == 0 or maze[x-1, y] == -1:
        values.append(-1)
    else:
        values.append(getFrom[x-2, y])
  # 3: RIGHT
    if x == MATRIX_SIZE-1 or maze[x+1, y] == -1:
        values.append(-1)
    else:
        values.append(getFrom[x+2, y])

    values.append(getFrom[x, y])

    return max(values)

### get_next() for getting the next best action when testing

In [ ]:
def get_next(getFrom, x, y):
    values = []
    # 0: UP
    if x == MATRIX_SIZE-1 or maze[x+1, y] == -1:
        values.append(-1)
    else:
        values.append(getFrom[x+2, y])
    # 1: DOWN
    if x == 0  or maze[x-1, y] == -1:
        values.append(-1)
    else:
        values.append(getFrom[x-2, y])
    # 2: LEFT
    if y == 0 or maze[x, y-1] == -1:
        values.append(-1)
    else:
        values.append(getFrom[x, y-2])
    # 3: RIGHT
    if y == MATRIX_SIZE-1 or maze[x, y+1] == -1:
        values.append(-1)
    else:
        values.append(getFrom[x, y+2])    
    print("\tNeighbours of {}, {}: {}".format(x, y, values))
    
    return values.index(max(values))

### find_path() to solve the maze 

In [ ]:
def find_path(start, finish, Q):
    x, y = start
    steps = []
    
    while (x,y) != finish:
        action = get_next(Q, x, y)
        if action == 0:
            print("MOVE UP", end="\t\t")
            steps.append("UP")
            x += 2
        elif action == 1:
            print("MOVE DOWN", end="\t")
            steps.append("DOWN")
            x -= 2
        elif action == 2:
            print("MOVE LEFT", end="\t")
            steps.append("LEFT")
            y -= 2
        elif action == 3:
            print("MOVE RIGHT", end="\t")
            steps.append("RIGHT")
            y += 2
        print(" -> ({}, {})".format(int(y/2-1), int(x/2-1)), end = " ")
    print("\tDestination reached!\n")
    return steps

## Initializing the Q-table

In [ ]:
Q = np.matrix(np.zeros([MATRIX_SIZE, MATRIX_SIZE]))
gamma = 0.8

## Training the model

In [ ]:
# Updating the q-table 
print("\nUpdating the q-table...")

for p in range(3):
    for start, action_list in episodes:
        x, y = start
        
        for action in action_list:
            if action == 0:     # 0: UP
                Q[x, y] = maze[x, y] + gamma * get_max(Q, x+2, y)
                Q[x+1, y] = Q[x, y]
                if (x, y) != finish:
                    x, y = x+2, y
            elif action == 1:   # 1: DOWN
                Q[x, y] = maze[x, y] + gamma * get_max(Q, x-2, y)
                Q[x-1, y] = Q[x, y]
                if (x, y) != finish:
                    x, y = x-2, y
            elif action == 2:   # 2: LEFT
                Q[x, y] = maze[x, y] + gamma * get_max(Q, x, y-2)
                Q[x, y-1] = Q[x, y]
                if (x, y) != finish:
                    x, y = x, y-2
            elif action == 3:   # 3: RIGHT
                Q[x, y] = maze[x, y] + gamma * get_max(Q, x, y+2)
                Q[x, y+1] = Q[x, y]
                if (x, y) != finish:
                    x, y = x, y+2
            if(x,y) == finish:  # If the agent reaches the finish state
                Q[x,y] += maze[x,y]

            # print("Start: {} \nQ-table: \n{} \n".format(start, Q))
            
    Q = Q/np.max(Q)*100  

print("\nQ-table: \n{} \n".format(Q))


Updating the q-table...

Q-table: 
[[  0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.  12.   0.   0.   0.   0.   0.   0.]
 [  0.   0.  12.   0.   0.   0.   0.   0.   0.]
 [  0.   0.  16.  16.  60.  60.  60.  60. 100.]
 [  0.   0.  16.   0.  32.   0.  40.   0.   0.]
 [  0.   0.  16.   0.  32.  16.  40.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.]] 



## Testing the model

In [ ]:
print("\nFinding the path...")
start = (2,2)
print("Start: ({}, {})".format(start[1]//2-1, start[0]//2-1))
print("Finish: ({}, {})\n".format(finish[1]//2-1, finish[0]//2-1))


print("({},{})".format(start[1]//2-1, start[0]//2-1), end=" ")
print(find_path(start, finish, Q))


Finding the path...
Start: (0, 0)
Finish: (3, 1)

(0,0) 	Neighbours of 2, 2: [16.0, -1, -1, -1]
MOVE UP		 -> (0, 1) 	Neighbours of 4, 2: [16.0, 12.0, -1, 60.0]
MOVE RIGHT	 -> (1, 1) 	Neighbours of 4, 4: [32.0, -1, 16.0, 60.0]
MOVE RIGHT	 -> (2, 1) 	Neighbours of 4, 6: [40.0, 0.0, 60.0, 100.0]
MOVE RIGHT	 -> (3, 1) 	Destination reached!

['UP', 'RIGHT', 'RIGHT', 'RIGHT']
